In [2]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import merge_config as config
import assay_sample_config as aconfig
import spectral_config as sconfig
import numpy as np
import math
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()
from tqdm import tqdm
import os
import file_config as fconfig
import pandas as pd

check for pandas
pandas is properly installed
check for tqdm
tqdm is properly installed
check for numpy
numpy is properly installed


In [7]:
file='/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Lab/XRF/XRF Master.xlsx'
sheets=pd.ExcelFile('/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Lab/XRF/XRF Master.xlsx')
output='/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Lab/XRF/_XRF Hole Masters/'


In [9]:
sheets.sheet_names

['readme',
 'Outdated Master',
 'Calibrations',
 'QAQC',
 'KM-22-73',
 'KM-22-76',
 'KM-22-77',
 'KM-22-78',
 'KM-22-80',
 'KM-22-81',
 'KM-22-82',
 'Template',
 'KM-21-25A',
 'KM-21-27',
 'KM-21-28',
 'KM-21-42']

In [10]:
for sheet in sheets.sheet_names:
    data=pd.read_excel(file,sheet_name=sheet)
    data.to_excel(output+sheet+'.xlsx')